# Modelado aditivo por etapas hacia adelante

**Forward stagewise additive modeling (FSAM)** optimiza la pérdida empı́rica con pérdida general y $f$ modelo aditivo. En la iteración $m$ calcula:
$$(\beta_m,\boldsymbol{\theta}_m)%
=\operatorname*{argmin}_{\beta,\boldsymbol{\theta}}\;L_m(\beta,\boldsymbol{\theta})%
\qquad\text{con}\qquad%
L_m(\beta,\boldsymbol{\theta})%
=\sum_{i=1}^N\ell(y_i, f_{m-1}(\boldsymbol{x}_i)+\beta F(\boldsymbol{x}_i;\boldsymbol{\theta}))$$
y reajusta el modelo,
$$f_m(\boldsymbol{x})=f_{m-1}(\boldsymbol{x})+\beta_m F_m(\boldsymbol{x})%
\qquad\text{con}\qquad%
F_m(\boldsymbol{x})=F(\boldsymbol{x};\boldsymbol{\theta}_m)$$

## Pérdida cuadrática y boosting mínimos cuadrados

Si $\ell(y,\hat{y})=(y-\hat{y})^2$, el objetivo en la iteración $m$ es:
$$L_m(\beta,\boldsymbol{\theta})%
=\sum_{i=1}^N (r_{im}-\beta F(\boldsymbol{x}_i;\boldsymbol{\theta}))^2
\qquad\text{con}\qquad r_{im}=y_i-f_{m-1}(\boldsymbol{x}_i)$$
**Boosting mínimos cuadrados** minimiza el objetivo fijando $\beta=1$ y ajustando $F$ a los residuos.

## Pérdida exponencial y AdaBoost

Si $\ell(\tilde{y},\hat{y})=\exp(-\tilde{y}\hat{y})$, $\tilde{y}\in\{-1,+1\}$, el objetivo en la iteración $m$ es:
$$\begin{align*}
L_m(\beta,\boldsymbol{\theta})%
&=\sum_{i=1}^N \exp(-\tilde{y}_i(f_{m-1}(\boldsymbol{x}_i)+\beta F(\boldsymbol{x}_i;\boldsymbol{\theta})))\\%
&=\sum_{i=1}^N w_{im}\exp(-\beta\tilde{y}_i F(\boldsymbol{x}_i;\boldsymbol{\theta}))%
\qquad\text{con}\qquad w_{im}=\exp(-\tilde{y}_i f_{m-1}(\boldsymbol{x}_i))\\%
&=e^{-\beta}\sum_{\tilde{y}_i=F(\boldsymbol{x}_i;\boldsymbol{\theta})} w_{im}%
+e^{\beta}\sum_{\tilde{y}_i\neq F(\boldsymbol{x}_i;\boldsymbol{\theta})} w_{im}\\%
&=(e^{\beta}-e^{-\beta})\sum_{i=1}^N w_{im}\mathbb{I}(\tilde{y}_i\neq F(\boldsymbol{x}_i;\boldsymbol{\theta}))%
+e^{-\beta}\sum_{i=1}^N w_{im}
\end{align*}$$
La minimización de $L_m$ puede hacerse en dos pasos. Primero hallamos $\boldsymbol{\theta}_m$ a partir de los datos ponderados:
$$\boldsymbol{\theta}_m=\operatorname*{argmin}_{\boldsymbol{\theta}}%
\sum_{i=1}^N w_{im}\mathbb{I}(\tilde{y}_i\neq F(\boldsymbol{x}_i;\boldsymbol{\theta}))$$
y luego $\beta_m$ mediante minimización en $\beta$ de $L_m(\beta,\boldsymbol{\theta}_m)$:
$$\beta_m=\operatorname*{argmin}_{\beta}\;L_m(\beta,\boldsymbol{\theta}_m)%
=\frac{1}{2}\log\frac{1-\operatorname{err}_m}{\operatorname{err}_m}%
\qquad\text{con}\qquad\operatorname{err}_m=\frac{1}{\sum_{i=1}^N w_{im}}%
\sum_{i=1}^N w_{im}\mathbb{I}(\tilde{y}_i\neq F_m(\boldsymbol{x}_i))$$
**Adaboost** halla $F_m(\cdot)$ y $\beta_m$ en la iteración $m$. Los pesos de los datos para la primera iteración son $w_{i1}=1/N$. Una vez hallados $F_m(\cdot)$ y $\beta_m$ en la iteración $m$, los pesos de los datos para la iteración $m+1$ se calculan fácilmente:
$$\begin{align*}
w_{i,m+1}&=\exp(-\tilde{y}_i f_m(\boldsymbol{x}_i))\\%
&=\exp(-\tilde{y}_i f_{m-1}(\boldsymbol{x}_i)-\tilde{y}_i\beta_m F_m(\boldsymbol{x}_i))\\%
&=w_{im}\exp(-\tilde{y}_i\beta_m F_m(\boldsymbol{x}_i))\\%
&=w_{im}\exp(\beta_m(2\mathbb{I}(\tilde{y}_i\neq F_m(\boldsymbol{x}_i))-1))\\%
&=w_{im}\exp(2\beta_m\mathbb{I}(\tilde{y}_i\neq F_m(\boldsymbol{x}_i)))\exp(-\beta_m)\\%
&=\begin{cases}
w_{im}\exp(2\beta_m) & \text{si $\tilde{y}_i\neq F_m(\boldsymbol{x}_i)$}\\%
w_{im}               & \text{en otro caso}%
\end{cases}
\end{align*}$$
El modelo ajustado para clasificación binaria es $f(\boldsymbol{x})=\operatorname{sgn}(\sum_m\beta_m F_m(\boldsymbol{x}))$; para regresión y clasificación multi-clase se usan variantes convenientemente adaptadas. En general, AdaBoost es muy sensible a outliers ya que los pesos de los datos mal clasificados crecen exponencialmente.

Otro aspecto que limita la aplicabilidad de AdaBoost es que no facilita la estimación probabilidades. En teoría, el riesgo de un modelo $f(\boldsymbol{x})$ con pérdida exponencial es:
$$\mathbb{E}[\exp(-\tilde{y}f(\boldsymbol{x}))\mid\boldsymbol{x}]%
=p(\tilde{y}=1\mid\boldsymbol{x})\exp(-f(\boldsymbol{x}))%
+p(\tilde{y}=-1\mid\boldsymbol{x})\exp(f(\boldsymbol{x}))$$
Derivando el riesgo con respecto a $f(\boldsymbol{x})$ e igualando a cero, obtenemos que el modelo de mínimo riesgo teórico halla la mitad de la log-odds:
$$f(\boldsymbol{x})=\frac{1}{2}\log\frac{p(\tilde{y}=1\mid\boldsymbol{x})}{p(\tilde{y}=-1\mid\boldsymbol{x})}$$
Nótese que este resultado justifica la aplicación del operador signo al modelo ajustado para clasificación binaria.

## Log-pérdida y LogitBoost

Con el fin de facilitar la estimación de probabilidades, el modelo aditivo puede usarse para predecir la mitad de la log-odds:
$$p(\tilde{y}\mid\boldsymbol{x};\boldsymbol{\theta})%
=\sigma(\tilde{y}a)\qquad\text{con}\qquad a=2f(\boldsymbol{x};\boldsymbol{\theta})$$
Si tomamos la log-pérdida, $\ell(\tilde{y},\boldsymbol{\theta};\boldsymbol{x})=-\log p(\tilde{y}\mid\boldsymbol{x};\boldsymbol{\theta})$, el objetivo en la iteración $m$ es, con $\beta=1$:
$$\begin{align*}
L_m(\boldsymbol{\theta})%
&=-\sum_{i=1}^N \log(\sigma(\tilde{y}_i2[f_{m-1}(\boldsymbol{x}_i)+ F(\boldsymbol{x}_i;\boldsymbol{\theta})]))\\%
&=\sum_{i=1}^N \log(1+\exp(-2\tilde{y}_i[f_{m-1}(\boldsymbol{x}_i)+ F(\boldsymbol{x}_i;\boldsymbol{\theta})]))
\end{align*}$$
**LogitBoost** es un algoritmo de Newton para minimizar este objetivo directamente.